In [1]:
# importing numpy and pandas
import numpy as np
import pandas as pd

In [2]:
# importing timesheet
timesheet = pd.read_excel("timesheet.xlsx",sheet_name="time")
timesheet.head()

,Name,emp id,In time,Out time,max late,max early
0,Abhishek,TIA009,11:00:00,20:30:00,11:10:00,20:20:00
1,Aditi,TIA019,10:00:00,18:30:00,10:10:00,18:20:00
2,Santosh,TIA028,09:00:00,18:00:00,09:10:00,17:50:00
3,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00
4,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00


In [3]:
# creating a dict of employee name with there id 
name_id = timesheet[["Name","emp id"]].set_index("Name").to_dict()['emp id']
print(name_id)

{'Abhishek': 'TIA009', 'Aditi': 'TIA019', 'Santosh': 'TIA028', 'Bantubala': 'Bantubala', 'Purnima': 'TIA024', 'Kuldeep': 'TIA013', 'Sachin': 'TIA012', 'Saheba': 'TIA004', 'Diksha': 'TIA027', 'Varun': 'Varun', 'Arvind': 'TIA026', 'Devesh': 'TIA023', 'Dev': 'Dev', 'Ajay': 'TIA011', 'Sakshee': 'Sakshee', 'Mahak': 'Mahak'}


In [4]:
# importing leave data - you can just copy and past the leave request data from google sheets directly to the leave sheet as it is
leave_data = pd.read_excel("leave_data.xlsx",sheet_name='leave')

# selecting only necessory column
leave_data =  leave_data[['Name','Leave Start Date','Leave End Date','Half Day/ Full Day?','Number of Days',"Type of leave"]]

# creating first name column
leave_data['first name'] = leave_data.apply(lambda x: x["Name"].split(' ')[0].strip(),axis=1)

leave_data['Leave End Date'] = pd.to_datetime(leave_data['Leave End Date'], errors='coerce')
leave_data['Leave Start Date'] = pd.to_datetime(leave_data['Leave Start Date'], errors='coerce')

leave_data = leave_data[leave_data['Leave Start Date'].notna()]
leave_data.head()

,Name,Leave Start Date,Leave End Date,Half Day/ Full Day?,Number of Days,Type of leave,first name
0,Anjali Bohra,2022-03-21,2022-03-21,Full day,1.0,Casual Leave,Anjali
1,Devendra Patidar,2022-03-24,2022-03-26,Full day,3.0,Sick leave (Illness or Injury),Devendra
2,Saheba Khan,2022-03-26,2022-03-26,Full day,1.0,Exam,Saheba
3,Saheba Khan,2022-03-29,2022-03-29,Full day,1.0,Exam,Saheba
4,Saheba Khan,2022-03-31,2022-03-31,Full day,1.0,Exam,Saheba


In [5]:
# fixing the structure of leave data table to long format
expanded_data = []

for _, row in leave_data.iterrows():
    start_date = row['Leave Start Date']
    end_date = row['Leave End Date']
    
    # Generate the date range
    dates = pd.date_range(start=start_date, end=end_date)
    
    for date in dates:
        expanded_data.append({
            'Name': row['Name'],
            "first name" : row["first name"],
            'Date': date,
            'Half Day/ Full Day?': row['Half Day/ Full Day?'],
            "Type of leave" : row['Type of leave']
        })

# Create the expanded DataFrame
expanded_df = pd.DataFrame(expanded_data)
leave_data = expanded_df

In [6]:
# creating leave columns

leave_data['is on full day leave'] = np.where(leave_data['Half Day/ Full Day?']=='Full day',1,0)
leave_data['is on half day leave'] = np.where(leave_data['Half Day/ Full Day?']!="Full day",1,0)
leave_data['is work from home'] = np.where(leave_data['Type of leave']=="Work From Home",1,0)
leave_data['is on full day leave'] = leave_data.apply(lambda x : 0 if x["is work from home"]==1 else x["is on full day leave"] ,axis=1)

leave_data['emp id'] = leave_data['first name'].apply(lambda x: name_id.get(x,x))
leave_data.to_excel("leaves.xlsx",index=False)

In [7]:
leave_data

,Name,first name,Date,Half Day/ Full Day?,Type of leave,is on full day leave,is on half day leave,is work from home,emp id
0,Anjali Bohra,Anjali,2022-03-21,Full day,Casual Leave,1,0,0,Anjali
1,Devendra Patidar,Devendra,2022-03-24,Full day,Sick leave (Illness or Injury),1,0,0,Devendra
2,Devendra Patidar,Devendra,2022-03-25,Full day,Sick leave (Illness or Injury),1,0,0,Devendra
3,Devendra Patidar,Devendra,2022-03-26,Full day,Sick leave (Illness or Injury),1,0,0,Devendra
4,Saheba Khan,Saheba,2022-03-26,Full day,Exam,1,0,0,TIA004
...,...,...,...,...,...,...,...,...,...
696,Purnima Khandelwal,Purnima,2025-05-05,Full day,Casual Leave,1,0,0,TIA024
697,Purnima Khandelwal,Purnima,2025-05-06,Full day,Casual Leave,1,0,0,TIA024
698,Aditi Chawda,Aditi,2025-05-07,Full day,Work From Home,0,0,1,TIA019
699,Purnima Khandelwal,Purnima,2025-05-07,Full day,Casual Leave,1,0,0,TIA024


In [8]:
holidays = pd.read_excel("holidays.xlsx")
holidays.head()

,date,holiday
0,2025-01-01,New Year
1,2025-01-14,Makar Sankranti
2,2025-01-26,Republic Day
3,2025-03-14,Holi
4,2025-03-19,Rang Panchmi


In [9]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     15 non-null     datetime64[ns]
 1   holiday  15 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 372.0+ bytes


In [10]:
# user id with there name data import and fix
# first working on user id data
user_data = open("user.dat","r").read()

# messy data in this file
print("messy data in this file\n",user_data[:800])
print("\n\ncleaned data ---> \n")
# the file contain non ACSII and symboles that we dont need we are only keeping the data with numbers and char
a = ""
for i in user_data:
    if i.isalnum():
        a += i
    else:
        a += " "
a = a.split()

# variable a contain a list of name,id,name,id,name,id so we need 2 list name and id 
name = [a[i] for i in range(len(a)) if i%2==0]
user_id = [int(a[i]) for i in range(len(a)) if i%2==1]
print("name- ",name)
print("User id -",user_id)
for i in range(len(name)):
    if name[i]=='Bantu':
        name[i] = "Bantubala"
# now converting this 2 list into one dictionary with id as key and name as value
user = dict(zip(user_id,name))
# print("Updated data in a dict",user)

# converting that dictionary to data frame
user_df = pd.DataFrame(user.values(),index=user.keys())
user_df.reset_index(inplace=True)
user_df.rename(columns={0:"Name","index":"id"},inplace=True)
user_df['emp id'] = user_df['Name'].apply(lambda x: name_id.get(x,np.nan))
user_df['emp id'] = user_df['emp id'].fillna(user_df['Name'])
# our work has done we get the data of users in this variable
user_df  



messy data in this file
           Saheba                              1                                 Lokesh                              2                                Hemant                              3                                Abhishek                            4                                 Kuldeep                             5                       	         Sachin                              9                       
         Varun                               10                                Priya                               11                                Bantubala                           12                      
          Saloni                              13                                Priyanka                            14                             


cleaned data ---> 

name-  ['Saheba', 'Lokesh', 'Hemant', 'Abhishek', 'Kuldeep', 'Sachin', 'Varun', 'Priya', 'Bantubala', 'Saloni', 'Priyanka', 'Govind', 'Purvi', 'Moiz', '

,id,Name,emp id
0,1,Saheba,TIA004
1,2,Lokesh,Lokesh
2,3,Hemant,Hemant
3,4,Abhishek,TIA009
4,5,Kuldeep,TIA013
5,9,Sachin,TIA012
6,10,Varun,Varun
7,11,Priya,Priya
8,12,Bantubala,Bantubala
9,13,Saloni,Saloni


In [62]:
# this is the main file which has the attendance data and reading it in csv
df = pd.read_csv("attendance.dat",header=None)

# because the data is in one column spliting it and keeping only the req. two columns so make these function that extract the data in first two columns

def get_id(text):
    return text.split("\t")[0]
def get_time(text):
    return text.split("\t")[1]

# creating two columns ID and punch_Time that hold the data when the employee punch in and out

df["id"]= df[0].map(get_id)
df["punch_time"] = df[0].map(get_time)
df.drop(columns=[0],inplace=True)   # deleting the raw data column and keeping only two column
df['id']= df["id"].astype(int)      # conerting id column to int

df.head()


,id,punch_time
0,23,2025-04-01 09:17:01
1,25,2025-04-01 09:25:18
2,28,2025-04-01 09:57:22
3,24,2025-04-01 09:59:49
4,1,2025-04-01 10:50:40


In [64]:
df['punch_time'] = pd.to_datetime(df['punch_time'])

In [68]:
df['punch_date'] = df['punch_time'].dt.date

In [69]:
df['punch_timeonly']= df['punch_time'].dt.time

In [74]:
df['punch_date'] = pd.to_datetime(df['punch_date'])

In [78]:
df['punch_timeonly'] = pd.to_datetime(df['punch_timeonly'],format = '%H:%M:%S')

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              519 non-null    int64         
 1   punch_time      519 non-null    datetime64[ns]
 2   punch_date      519 non-null    datetime64[ns]
 3   punch_timeonly  519 non-null    datetime64[ns]
dtypes: datetime64[ns](3), int64(1)
memory usage: 16.3 KB


In [81]:
df.head()

,id,punch_time,punch_date,punch_timeonly
0,23,2025-04-01 09:17:01,2025-04-01,1900-01-01 09:17:01
1,25,2025-04-01 09:25:18,2025-04-01,1900-01-01 09:25:18
2,28,2025-04-01 09:57:22,2025-04-01,1900-01-01 09:57:22
3,24,2025-04-01 09:59:49,2025-04-01,1900-01-01 09:59:49
4,1,2025-04-01 10:50:40,2025-04-01,1900-01-01 10:50:40


In [82]:
df.sort_values(by=['id','punch_time'],inplace=True)

In [88]:
df_intime = pd.DataFrame(df.groupby(['id','punch_date']).min()['punch_timeonly']).reset_index()

In [89]:
df_outtime = pd.DataFrame(df.groupby(['id','punch_date']).max()['punch_timeonly']).reset_index()

In [91]:
df_intime.rename(columns={'punch_timeonly':'punch_intime'},inplace=True)
df_outtime.rename(columns={'punch_timeonly':'punch_outtime'},inplace=True)

In [92]:
df_f1 = pd.merge(df_intime,df_outtime,on=['id','punch_date'],how='left')

In [93]:
df_f1

,id,punch_date,punch_intime,punch_outtime
0,1,2025-04-01,1900-01-01 10:50:40,1900-01-01 19:02:35
1,1,2025-04-02,1900-01-01 10:36:48,1900-01-01 19:09:44
2,1,2025-04-03,1900-01-01 10:23:39,1900-01-01 10:23:39
3,1,2025-04-04,1900-01-01 10:40:03,1900-01-01 18:56:18
4,1,2025-04-05,1900-01-01 10:40:27,1900-01-01 19:18:28
...,...,...,...,...
265,28,2025-04-21,1900-01-01 10:12:27,1900-01-01 17:37:35
266,28,2025-04-22,1900-01-01 10:05:57,1900-01-01 17:25:39
267,28,2025-04-23,1900-01-01 09:49:12,1900-01-01 17:30:27
268,28,2025-04-25,1900-01-01 09:51:45,1900-01-01 17:26:00


In [12]:
# merge the data of id and punch time data into one data frame
data = pd.merge(df,user_df,how="left",on="id")

In [13]:
# Combined data
data.head()

,id,punch_time,Name,emp id
0,23,2025-04-01 09:17:01,Devesh,TIA023
1,25,2025-04-01 09:25:18,Arvind,TIA026
2,28,2025-04-01 09:57:22,Mahak,Mahak
3,24,2025-04-01 09:59:49,Purnima,TIA024
4,1,2025-04-01 10:50:40,Saheba,TIA004


In [14]:
# creating date table
all_dates_in_data = pd.date_range(start=min(data['punch_time']),end=max(data['punch_time']))

all_dates_in_data = pd.to_datetime(all_dates_in_data)

all_dates_in_data = pd.DataFrame(pd.DataFrame(all_dates_in_data)[0].dt.date)

all_dates_in_data['date'] = pd.to_datetime(all_dates_in_data[0])

all_dates_in_data['week name'] = all_dates_in_data['date'].dt.strftime("%a")

all_dates_in_data

,0,date,week name
0,2025-04-01,2025-04-01,Tue
1,2025-04-02,2025-04-02,Wed
2,2025-04-03,2025-04-03,Thu
3,2025-04-04,2025-04-04,Fri
4,2025-04-05,2025-04-05,Sat
5,2025-04-06,2025-04-06,Sun
6,2025-04-07,2025-04-07,Mon
7,2025-04-08,2025-04-08,Tue
8,2025-04-09,2025-04-09,Wed
9,2025-04-10,2025-04-10,Thu


In [15]:
all_dates_in_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   0          30 non-null     object        
 1   date       30 non-null     datetime64[ns]
 2   week name  30 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 852.0+ bytes


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          519 non-null    int64 
 1   punch_time  519 non-null    object
 2   Name        519 non-null    object
 3   emp id      519 non-null    object
dtypes: int64(1), object(3)
memory usage: 16.3+ KB


In [17]:
# extracting date and time 

data['datetime'] = pd.to_datetime(df['punch_time'])
data['date'] = pd.to_datetime(df['punch_time']).dt.date
data['in_out_time'] = pd.to_datetime(df['punch_time']).dt.time
data["am_pm"] = data['datetime'].dt.strftime("%p")
data.drop(columns="punch_time",inplace=True)  
data['date'] = pd.to_datetime(data['date'])
data.head()

,id,Name,emp id,datetime,date,in_out_time,am_pm
0,23,Devesh,TIA023,2025-04-01 09:17:01,2025-04-01,09:17:01,AM
1,25,Arvind,TIA026,2025-04-01 09:25:18,2025-04-01,09:25:18,AM
2,28,Mahak,Mahak,2025-04-01 09:57:22,2025-04-01,09:57:22,AM
3,24,Purnima,TIA024,2025-04-01 09:59:49,2025-04-01,09:59:49,AM
4,1,Saheba,TIA004,2025-04-01 10:50:40,2025-04-01,10:50:40,AM


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           519 non-null    int64         
 1   Name         519 non-null    object        
 2   emp id       519 non-null    object        
 3   datetime     519 non-null    datetime64[ns]
 4   date         519 non-null    datetime64[ns]
 5   in_out_time  519 non-null    object        
 6   am_pm        519 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 28.5+ KB


In [19]:
data = pd.merge(all_dates_in_data,data,how="left",left_on="date",right_on='date')

In [20]:
# merge time data to main dataframe

data = pd.merge(data,timesheet,how="left",on ='Name')
data.head()

,0,date,week name,id,Name,emp id_x,datetime,in_out_time,am_pm,emp id_y,In time,Out time,max late,max early
0,2025-04-01,2025-04-01,Tue,23.0,Devesh,TIA023,2025-04-01 09:17:01,09:17:01,AM,TIA023,09:30:00,18:00:00,09:40:00,17:50:00
1,2025-04-01,2025-04-01,Tue,25.0,Arvind,TIA026,2025-04-01 09:25:18,09:25:18,AM,TIA026,09:30:00,18:00:00,09:40:00,17:50:00
2,2025-04-01,2025-04-01,Tue,28.0,Mahak,Mahak,2025-04-01 09:57:22,09:57:22,AM,Mahak,10:00:00,17:30:00,10:10:00,17:20:00
3,2025-04-01,2025-04-01,Tue,24.0,Purnima,TIA024,2025-04-01 09:59:49,09:59:49,AM,TIA024,10:00:00,18:30:00,10:10:00,18:20:00
4,2025-04-01,2025-04-01,Tue,1.0,Saheba,TIA004,2025-04-01 10:50:40,10:50:40,AM,TIA004,11:00:00,18:30:00,11:10:00,18:20:00


In [21]:
# data.to_excel("aaaamain_data.xlsx",index=False)

In [22]:
data.head()

,0,date,week name,id,Name,emp id_x,datetime,in_out_time,am_pm,emp id_y,In time,Out time,max late,max early
0,2025-04-01,2025-04-01,Tue,23.0,Devesh,TIA023,2025-04-01 09:17:01,09:17:01,AM,TIA023,09:30:00,18:00:00,09:40:00,17:50:00
1,2025-04-01,2025-04-01,Tue,25.0,Arvind,TIA026,2025-04-01 09:25:18,09:25:18,AM,TIA026,09:30:00,18:00:00,09:40:00,17:50:00
2,2025-04-01,2025-04-01,Tue,28.0,Mahak,Mahak,2025-04-01 09:57:22,09:57:22,AM,Mahak,10:00:00,17:30:00,10:10:00,17:20:00
3,2025-04-01,2025-04-01,Tue,24.0,Purnima,TIA024,2025-04-01 09:59:49,09:59:49,AM,TIA024,10:00:00,18:30:00,10:10:00,18:20:00
4,2025-04-01,2025-04-01,Tue,1.0,Saheba,TIA004,2025-04-01 10:50:40,10:50:40,AM,TIA004,11:00:00,18:30:00,11:10:00,18:20:00


In [23]:
data[data['week name']=='Sun']

,0,date,week name,id,Name,emp id_x,datetime,in_out_time,am_pm,emp id_y,In time,Out time,max late,max early
97,2025-04-06,2025-04-06,Sun,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,2025-04-13,2025-04-13,Sun,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,2025-04-20,2025-04-20,Sun,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465,2025-04-27,2025-04-27,Sun,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
data

,0,date,week name,id,Name,emp id_x,datetime,in_out_time,am_pm,emp id_y,In time,Out time,max late,max early
0,2025-04-01,2025-04-01,Tue,23.0,Devesh,TIA023,2025-04-01 09:17:01,09:17:01,AM,TIA023,09:30:00,18:00:00,09:40:00,17:50:00
1,2025-04-01,2025-04-01,Tue,25.0,Arvind,TIA026,2025-04-01 09:25:18,09:25:18,AM,TIA026,09:30:00,18:00:00,09:40:00,17:50:00
2,2025-04-01,2025-04-01,Tue,28.0,Mahak,Mahak,2025-04-01 09:57:22,09:57:22,AM,Mahak,10:00:00,17:30:00,10:10:00,17:20:00
3,2025-04-01,2025-04-01,Tue,24.0,Purnima,TIA024,2025-04-01 09:59:49,09:59:49,AM,TIA024,10:00:00,18:30:00,10:10:00,18:20:00
4,2025-04-01,2025-04-01,Tue,1.0,Saheba,TIA004,2025-04-01 10:50:40,10:50:40,AM,TIA004,11:00:00,18:30:00,11:10:00,18:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,2025-04-30,2025-04-30,Wed,23.0,Devesh,TIA023,2025-04-30 18:27:49,18:27:49,PM,TIA023,09:30:00,18:00:00,09:40:00,17:50:00
519,2025-04-30,2025-04-30,Wed,20.0,Aditi,TIA019,2025-04-30 18:39:42,18:39:42,PM,TIA019,10:00:00,18:30:00,10:10:00,18:20:00
520,2025-04-30,2025-04-30,Wed,5.0,Kuldeep,TIA013,2025-04-30 18:53:03,18:53:03,PM,TIA013,11:00:00,18:00:00,11:10:00,17:50:00
521,2025-04-30,2025-04-30,Wed,6.0,Sakshee,Sakshee,2025-04-30 18:57:03,18:57:03,PM,Sakshee,10:00:00,19:00:00,10:10:00,18:50:00


In [25]:
test = data.pivot_table(index=[
 'date',
 'week name',
 'id',
 'Name',
 'emp id_x',
 'In time',
 'Out time',
 'max late',
 'max early'],columns='am_pm',values='in_out_time',aggfunc=['min','max'])

In [26]:
test.reset_index()

date week name    id       Name   emp id_x   In time  Out time  \
am_pm                                                                        
0     2025-04-01       Tue   1.0     Saheba     TIA004  11:00:00  18:30:00   
1     2025-04-01       Tue   5.0    Kuldeep     TIA013  11:00:00  18:00:00   
2     2025-04-01       Tue  12.0  Bantubala  Bantubala  11:30:00  15:00:00   
3     2025-04-01       Tue  23.0     Devesh     TIA023  09:30:00  18:00:00   
4     2025-04-01       Tue  24.0    Purnima     TIA024  10:00:00  18:30:00   
..           ...       ...   ...        ...        ...       ...       ...   
263   2025-04-30       Wed  27.0    Santosh     TIA028  09:00:00  18:00:00   
264   2025-04-30       Wed   4.0   Abhishek     TIA009  11:00:00  20:30:00   
265   2025-04-30       Wed  10.0      Varun      Varun  09:30:00  18:00:00   
266   2025-04-30       Wed  20.0      Aditi     TIA019  10:00:00  18:30:00   
267   2025-04-30       Wed   6.0    Sakshee    Sakshee  10:00:00  19:00:00   

       max late max early       min                 max            
am_pm                            AM        PM        AM        PM  
0      11:10:00  18:20:00  10:50:40  19:02:35  10:51:34  19:02:35  
1      11:10:00  17:50:00  10:58:09  18:53:30  10:58:09  18:53:30  
2      11:40:00  14:50:00  11:01:31  15:04:41  11:01:31  15:06:11  
3      09:40:00  17:50:00  09:17:01  18:30:02  09:17:01  18:30:02  
4      10:10:00  18:20:00  09:59:49  18:53:43  09:59:49  18:53:43  
..          ...       ...       ...       ...       ...       ...  
263    09:10:00  17:50:00  08:22:28  17:57:18  08:22:28  17:57:18  
264    11:10:00  20:20:00  10:53:09       NaN  10:53:09       NaN  
265    09:40:00  17:50:00  09:45:37       NaN  09:45:39       NaN  
266    10:10:00  18:20:00  09:26:35  18:39:42  09:26:35  18:39:42  
267    10:10:00  18:50:00       NaN  18:57:03       NaN  18:57:03  

[268 rows x 13 columns]

In [27]:
fdata = test['min'].reset_index()
fdata

am_pm,date,week name,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM
0,2025-04-01,Tue,1.0,Saheba,TIA004,11:00:00,18:30:00,11:10:00,18:20:00,10:50:40,19:02:35
1,2025-04-01,Tue,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:58:09,18:53:30
2,2025-04-01,Tue,12.0,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00,11:01:31,15:04:41
3,2025-04-01,Tue,23.0,Devesh,TIA023,09:30:00,18:00:00,09:40:00,17:50:00,09:17:01,18:30:02
4,2025-04-01,Tue,24.0,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00,09:59:49,18:53:43
...,...,...,...,...,...,...,...,...,...,...,...
263,2025-04-30,Wed,27.0,Santosh,TIA028,09:00:00,18:00:00,09:10:00,17:50:00,08:22:28,17:57:18
264,2025-04-30,Wed,4.0,Abhishek,TIA009,11:00:00,20:30:00,11:10:00,20:20:00,10:53:09,NaN
265,2025-04-30,Wed,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:45:37,NaN
266,2025-04-30,Wed,20.0,Aditi,TIA019,10:00:00,18:30:00,10:10:00,18:20:00,09:26:35,18:39:42


In [28]:
a = test['max'].reset_index()
a

am_pm,date,week name,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM
0,2025-04-01,Tue,1.0,Saheba,TIA004,11:00:00,18:30:00,11:10:00,18:20:00,10:51:34,19:02:35
1,2025-04-01,Tue,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:58:09,18:53:30
2,2025-04-01,Tue,12.0,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00,11:01:31,15:06:11
3,2025-04-01,Tue,23.0,Devesh,TIA023,09:30:00,18:00:00,09:40:00,17:50:00,09:17:01,18:30:02
4,2025-04-01,Tue,24.0,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00,09:59:49,18:53:43
...,...,...,...,...,...,...,...,...,...,...,...
263,2025-04-30,Wed,27.0,Santosh,TIA028,09:00:00,18:00:00,09:10:00,17:50:00,08:22:28,17:57:18
264,2025-04-30,Wed,4.0,Abhishek,TIA009,11:00:00,20:30:00,11:10:00,20:20:00,10:53:09,NaN
265,2025-04-30,Wed,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:45:39,NaN
266,2025-04-30,Wed,20.0,Aditi,TIA019,10:00:00,18:30:00,10:10:00,18:20:00,09:26:35,18:39:42


In [29]:
fdata['PM'] = a['PM']

In [30]:
fdata.head()

am_pm,date,week name,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM
0,2025-04-01,Tue,1.0,Saheba,TIA004,11:00:00,18:30:00,11:10:00,18:20:00,10:50:40,19:02:35
1,2025-04-01,Tue,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:58:09,18:53:30
2,2025-04-01,Tue,12.0,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00,11:01:31,15:06:11
3,2025-04-01,Tue,23.0,Devesh,TIA023,09:30:00,18:00:00,09:40:00,17:50:00,09:17:01,18:30:02
4,2025-04-01,Tue,24.0,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00,09:59:49,18:53:43


In [31]:
fdata[fdata['AM']>=fdata['max late']]

am_pm,date,week name,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM
27,2025-04-03,Thu,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:48:44,NaN
105,2025-04-12,Sat,28.0,Mahak,Mahak,10:00:00,17:30:00,10:10:00,17:20:00,10:13:59,17:27:16
107,2025-04-12,Sat,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:44:51,NaN
150,2025-04-17,Thu,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:41:30,18:20:52
161,2025-04-18,Fri,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:45:14,18:25:08
171,2025-04-19,Sat,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:48:21,13:48:44
179,2025-04-21,Mon,28.0,Mahak,Mahak,10:00:00,17:30:00,10:10:00,17:20:00,10:12:27,17:37:35
181,2025-04-21,Mon,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:44:28,18:19:47
235,2025-04-26,Sat,6.0,Sakshee,Sakshee,10:00:00,19:00:00,10:10:00,18:50:00,10:19:06,NaN
255,2025-04-29,Tue,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:46:08,19:31:53


In [32]:
# filling null values where data is not present for punch in or punch out
fdata['AM'] = fdata['AM'].fillna(fdata['In time'])
fdata['PM'] = fdata['PM'].fillna(fdata['Out time'])

In [33]:
fdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       268 non-null    datetime64[ns]
 1   week name  268 non-null    object        
 2   id         268 non-null    float64       
 3   Name       268 non-null    object        
 4   emp id_x   268 non-null    object        
 5   In time    268 non-null    object        
 6   Out time   268 non-null    object        
 7   max late   268 non-null    object        
 8   max early  268 non-null    object        
 9   AM         268 non-null    object        
 10  PM         268 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 23.2+ KB


In [34]:
fdata.sample(5)

am_pm,date,week name,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM
265,2025-04-30,Wed,10.0,Varun,Varun,09:30:00,18:00:00,09:40:00,17:50:00,09:45:37,18:00:00
90,2025-04-11,Fri,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:57:29,18:55:43
82,2025-04-10,Thu,23.0,Devesh,TIA023,09:30:00,18:00:00,09:40:00,17:50:00,09:21:07,18:26:37
83,2025-04-10,Thu,24.0,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00,09:27:45,18:24:29
59,2025-04-08,Tue,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:54:34,18:53:11


In [35]:
fdata['come late'] = np.where(fdata['AM']>=fdata['max late'],1,0)
fdata['early leave'] = np.where(fdata['PM']<=fdata['max early'],1,0)

In [36]:
fdata.head()

am_pm,date,week name,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM,come late,early leave
0,2025-04-01,Tue,1.0,Saheba,TIA004,11:00:00,18:30:00,11:10:00,18:20:00,10:50:40,19:02:35,0,0
1,2025-04-01,Tue,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:58:09,18:53:30,0,0
2,2025-04-01,Tue,12.0,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00,11:01:31,15:06:11,0,0
3,2025-04-01,Tue,23.0,Devesh,TIA023,09:30:00,18:00:00,09:40:00,17:50:00,09:17:01,18:30:02,0,0
4,2025-04-01,Tue,24.0,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00,09:59:49,18:53:43,0,0


In [37]:
fdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         268 non-null    datetime64[ns]
 1   week name    268 non-null    object        
 2   id           268 non-null    float64       
 3   Name         268 non-null    object        
 4   emp id_x     268 non-null    object        
 5   In time      268 non-null    object        
 6   Out time     268 non-null    object        
 7   max late     268 non-null    object        
 8   max early    268 non-null    object        
 9   AM           268 non-null    object        
 10  PM           268 non-null    object        
 11  come late    268 non-null    int64         
 12  early leave  268 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory usage: 27.3+ KB


In [38]:
fdata = pd.merge(all_dates_in_data,fdata,how="left",left_on="date",right_on='date')

In [39]:
fdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   0            272 non-null    object        
 1   date         272 non-null    datetime64[ns]
 2   week name_x  272 non-null    object        
 3   week name_y  268 non-null    object        
 4   id           268 non-null    float64       
 5   Name         268 non-null    object        
 6   emp id_x     268 non-null    object        
 7   In time      268 non-null    object        
 8   Out time     268 non-null    object        
 9   max late     268 non-null    object        
 10  max early    268 non-null    object        
 11  AM           268 non-null    object        
 12  PM           268 non-null    object        
 13  come late    268 non-null    float64       
 14  early leave  268 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(11)
memory usage

In [40]:
# creating unique id in leave data
leave_data['unique id'] = leave_data['Date'].astype(str)+"-"+leave_data['first name']

In [41]:
# creating unique id in fdata
fdata["unique id"] = fdata['date'].astype(str)+"-"+fdata['Name']
fdata.head()

,0,date,week name_x,week name_y,id,Name,emp id_x,In time,Out time,max late,max early,AM,PM,come late,early leave,unique id
0,2025-04-01,2025-04-01,Tue,Tue,1.0,Saheba,TIA004,11:00:00,18:30:00,11:10:00,18:20:00,10:50:40,19:02:35,0.0,0.0,2025-04-01-Saheba
1,2025-04-01,2025-04-01,Tue,Tue,5.0,Kuldeep,TIA013,11:00:00,18:00:00,11:10:00,17:50:00,10:58:09,18:53:30,0.0,0.0,2025-04-01-Kuldeep
2,2025-04-01,2025-04-01,Tue,Tue,12.0,Bantubala,Bantubala,11:30:00,15:00:00,11:40:00,14:50:00,11:01:31,15:06:11,0.0,0.0,2025-04-01-Bantubala
3,2025-04-01,2025-04-01,Tue,Tue,23.0,Devesh,TIA023,09:30:00,18:00:00,09:40:00,17:50:00,09:17:01,18:30:02,0.0,0.0,2025-04-01-Devesh
4,2025-04-01,2025-04-01,Tue,Tue,24.0,Purnima,TIA024,10:00:00,18:30:00,10:10:00,18:20:00,09:59:49,18:53:43,0.0,0.0,2025-04-01-Purnima


In [42]:
an = list(fdata['Name'].unique())

In [43]:
an.remove(np.nan)


In [44]:
# cretating a combine date and emp name key
all_emp = list(an)

all_dates = list(all_dates_in_data['date'].astype(str))

In [45]:
temp = []
for i in all_dates:
    for j in all_emp:
        temp.append(str(i)+"-"+str(j))
    

temp

['2025-04-01-Saheba',
 '2025-04-01-Kuldeep',
 '2025-04-01-Bantubala',
 '2025-04-01-Devesh',
 '2025-04-01-Purnima',
 '2025-04-01-Arvind',
 '2025-04-01-Santosh',
 '2025-04-01-Mahak',
 '2025-04-01-Abhishek',
 '2025-04-01-Varun',
 '2025-04-01-Aditi',
 '2025-04-01-Sakshee',
 '2025-04-02-Saheba',
 '2025-04-02-Kuldeep',
 '2025-04-02-Bantubala',
 '2025-04-02-Devesh',
 '2025-04-02-Purnima',
 '2025-04-02-Arvind',
 '2025-04-02-Santosh',
 '2025-04-02-Mahak',
 '2025-04-02-Abhishek',
 '2025-04-02-Varun',
 '2025-04-02-Aditi',
 '2025-04-02-Sakshee',
 '2025-04-03-Saheba',
 '2025-04-03-Kuldeep',
 '2025-04-03-Bantubala',
 '2025-04-03-Devesh',
 '2025-04-03-Purnima',
 '2025-04-03-Arvind',
 '2025-04-03-Santosh',
 '2025-04-03-Mahak',
 '2025-04-03-Abhishek',
 '2025-04-03-Varun',
 '2025-04-03-Aditi',
 '2025-04-03-Sakshee',
 '2025-04-04-Saheba',
 '2025-04-04-Kuldeep',
 '2025-04-04-Bantubala',
 '2025-04-04-Devesh',
 '2025-04-04-Purnima',
 '2025-04-04-Arvind',
 '2025-04-04-Santosh',
 '2025-04-04-Mahak',
 '2025-04

In [46]:
temp = pd.DataFrame(temp)

In [47]:
combination_of_emp_name_with_dates = temp

In [48]:
#-- ----->Now i ony need to combine all the data using all combination of emp name  with dates to the leave data and fdata and then fill all column names, keep neccessory columns and then export the data

In [49]:
a = pd.merge(combination_of_emp_name_with_dates,fdata,how="left",left_on=0,right_on="unique id")

In [50]:
a = pd.merge(a,leave_data,how="left",left_on='0_x',right_on="unique id")

In [51]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   key_0                 360 non-null    object        
 1   0_x                   360 non-null    object        
 2   0_y                   268 non-null    object        
 3   date                  268 non-null    datetime64[ns]
 4   week name_x           268 non-null    object        
 5   week name_y           268 non-null    object        
 6   id                    268 non-null    float64       
 7   Name_x                268 non-null    object        
 8   emp id_x              268 non-null    object        
 9   In time               268 non-null    object        
 10  Out time              268 non-null    object        
 11  max late              268 non-null    object        
 12  max early             268 non-null    object        
 13  AM                  

In [52]:
a.head(1).T

,0
key_0,2025-04-01-Saheba
0_x,2025-04-01-Saheba
0_y,2025-04-01
date,2025-04-01 00:00:00
week name_x,Tue
week name_y,Tue
id,1.0
Name_x,Saheba
emp id_x,TIA004
In time,11:00:00


In [53]:
# keeping columns we need
df= a[['key_0','id','Name_x','emp id_x','In time','Out time','AM','PM','come late','early leave','is on full day leave','is on half day leave','is work from home']]

In [54]:
df['date'] = pd.to_datetime(df['key_0'].apply(lambda x : x[:10]))

C:\Users\deves\AppData\Local\Temp\ipykernel_7152\2889867911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['key_0'].apply(lambda x : x[:10]))


In [55]:
new_name = ['unique key',
 'biometric id',
 'name',
 'employee id',
 'official in time',
 'official out time',
 'actual in time',
 'actual out time',
 'come late',
 'leave early',
 'is on full day leave',
 'is on half day leave',
 'is work from home',
 'date']

In [56]:
df.rename(columns=dict(zip(df.columns,new_name)),inplace=True)

C:\Users\deves\AppData\Local\Temp\ipykernel_7152\3528898841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=dict(zip(df.columns,new_name)),inplace=True)


In [57]:
df['week day'] = df['date'].dt.strftime("%A")

C:\Users\deves\AppData\Local\Temp\ipykernel_7152\356131702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week day'] = df['date'].dt.strftime("%A")


In [58]:
df['name'] = df['unique key'].apply(lambda x: x[11:])

C:\Users\deves\AppData\Local\Temp\ipykernel_7152\62801480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['unique key'].apply(lambda x: x[11:])


In [59]:
df['employee id'] =  df['name'].apply(lambda x : name_id[x])

C:\Users\deves\AppData\Local\Temp\ipykernel_7152\125135562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['employee id'] =  df['name'].apply(lambda x : name_id[x])


In [61]:
# appending the data which we clean to the all data file
try :
    filedf = pd.read_excel("main_data.xlsx")
except:
    pd.DataFrame().to_excel("main_data.xlsx")
    filedf = pd.read_excel("main_data.xlsx")
filedf = pd.concat((filedf,df))
filedf = filedf.drop_duplicates()
filedf.to_excel("main_data.xlsx",index=False)